# Strava API Pipeline Workbook

## Basic Setup

In [1]:
import requests
import json
import datetime
from datetime import datetime
from datetime import date
import time
import pandas as pd
import os
from dotenv import load_dotenv
import sqlite3

pd.set_option('display.max_columns', None)

# Developer Docs
# https://developers.strava.com/docs/getting-started/#basic

# Streams API Details
# https://developers.strava.com/docs/reference/#api-Streams-getActivityStreams

# Replace with your actual credentials
load_dotenv(dotenv_path="secrets.env")
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
REFRESH_TOKEN = os.environ.get("REFRESH_TOKEN")

DB_PATH = 'strava_data.db'

load_dotenv(dotenv_path="secrets.env", override=True)

True

In [35]:
DB_PATH = 'strava_data.db'

In [3]:
print(CLIENT_ID)

64543


### Error Troubleshooting

HTTP error occurred: 401 Client Error: Unauthorized for url: https://www.strava.com/api/v3/athlete/activities?page=1&per_page=1

Go to this link, and note the scopes being used at the end of the URL - 
https://www.strava.com/oauth/authorize?client_id=64543&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all

More details on scope here - https://developers.strava.com/docs/authentication/#detailsaboutrequestingaccess

Extract the auth code from the reply URL and assign it using the below cell.

Then, run the contents of the Code to Exchange Auth Code for Auth Token section to get a valid AUTH_TOKEN

In [ ]:
AUTH_CODE = "cf2cfc3a2fda1c8b9f2d6ed48623175047d3ea29"

## How this SHOULD work

Go to this link, and note the scopes being used at the end of the URL - 
https://www.strava.com/oauth/authorize?client_id=64543&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all

In [4]:
AUTH_CODE = 'cf2cfc3a2fda1c8b9f2d6ed48623175047d3ea29'

The auth code provides the initial access and scope to the app's athletes but is not used for queries.

That AUTH CODE should then be used below to exchange for a short lived (6 hours?) ACCESS TOKEN

We will also store the replied REFRESH_TOKEN in the environment variables

In [5]:
def get_auth_token(client_id, client_secret, auth_code):
    """Refreshes the access token using the refresh token."""
    response = requests.post(
        url="https://www.strava.com/oauth/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "grant_type": "authorization_code",
            "code": auth_code,
        },
    )
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return response.json()

try:
    access_token_request = get_auth_token(CLIENT_ID, CLIENT_SECRET, AUTH_CODE)
    ACCESS_TOKEN = access_token_request["access_token"]
    os.environ["REFRESH_TOKEN"] = access_token_request["refresh_token"]

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    
print(os.environ.get("REFRESH_TOKEN"))
print(access_token_request)
print(os.environ.get("REFRESH_TOKEN"))

5d4dc1fd084e0994631899facefef89478de90d3
{'token_type': 'Bearer', 'expires_at': 1747260261, 'expires_in': 21600, 'refresh_token': '5d4dc1fd084e0994631899facefef89478de90d3', 'access_token': 'c3f5b56134d8a3b061cc58ee0ae47571b61a5f93', 'athlete': {'id': 24266563, 'username': 'smithcharlie', 'resource_state': 2, 'firstname': 'Charlie', 'lastname': 'Smith 🦚', 'bio': '', 'city': 'Minneapolis', 'state': 'MN', 'country': 'United States', 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2017-08-17T15:31:27Z', 'updated_at': '2025-04-19T18:19:00Z', 'badge_type_id': 0, 'weight': 63.5029, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/24266563/11743602/9/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/24266563/11743602/9/large.jpg', 'friend': None, 'follower': None}}
5d4dc1fd084e0994631899facefef89478de90d3


In [6]:
os.environ["ACCESS_TOKEN"] = ACCESS_TOKEN

In [7]:
print(ACCESS_TOKEN)

ASDFASDASDFADFDFA


In [9]:
import utils.strava_utils as strava_utils
ACCESS_TOKEN = "c3f5b56134d8a3b061cc58ee0ae47571b61a5f93"
strava_utils.update_env_variable("ACCESS_TOKEN", ACCESS_TOKEN)

ACCESS_TOKEN


THEN once that ACCESS TOKEN expires, we can try to exchange for a new one using the refresh code we were provided and the below code

## Refresh Auth Token

The manual cell below is necessary because the key from the prior process is stored in an environment variable. This should work for the airflow approach but will need to be copied manually when running local.

In [93]:
REFRESH_TOKEN = "5d4dc1fd084e0994631899facefef89478de90d3"
print(REFRESH_TOKEN)

5d4dc1fd084e0994631899facefef89478de90d3


In [138]:
# old gemini code don't know if useful or not
# This SHOULD work 
def refresh_access_token(client_id, client_secret, refresh_token):
    """Refreshes the access token using the refresh token."""
    response = requests.post(
        url="https://www.strava.com/oauth/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
        },
    )
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    os.environ["REFRESH_TOKEN"] = response["refresh_token"]
    return response.json()["access_token"]

In [139]:
try:
    refresh_attempt = refresh_access_token(CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)
    ACCESS_TOKEN = refresh_attempt["access_token"]
    os.environ["REFRESH_TOKEN"] = access_token_request["refresh_token"]
    

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

HTTP error occurred: 401 Client Error: Unauthorized for url: https://www.strava.com/oauth/token


In [140]:
refresh_attempt

{'token_type': 'Bearer',
 'access_token': 'd49fc739be1f780349f472f3b88a971ea6eb9843',
 'expires_at': 1747020565,
 'expires_in': 21600,
 'refresh_token': '5d4dc1fd084e0994631899facefef89478de90d3'}

## Get Auth Token
#### The Auth Code input here should be the code extracted from the blank webpage after a user approves a scope request

In [ ]:
def get_auth_token(client_id, client_secret, auth_code):
    """Refreshes the access token using the refresh token."""
    response = requests.post(
        url="https://www.strava.com/oauth/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "grant_type": "authorization_code",
            "code": auth_code,
        },
    )
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return response.json()

#### Code to Exchange Auth Code for Auth Token

In [61]:
try:
    swap_auth_token_attempt = get_auth_token(CLIENT_ID, CLIENT_SECRET, AUTH_CODE)
    AUTH_TOKEN = swap_auth_token_attempt["access_token"]

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    
print(swap_auth_token_attempt)

An unexpected error occurred: 'access_token'
{'message': 'Bad Request', 'errors': [{'resource': 'AuthorizationCode', 'field': 'code', 'code': 'invalid'}]}


## Base Functions for retreiving activities and timeseries data from Strava API

In [10]:
def get_activities(access_token, page=1, per_page=30, **optional_parameters):
    """Retrieves activities from the Strava API.
    Optional parameters should be provided at the end of the call like so:
    before = epoch_timestamp, after = epoch_timestamp
    """
    url = f"https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"page": page, "per_page": per_page}
    params.update(optional_parameters)
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_streams(access_token, keys, activity_id):
    """Retrieves activities from the Strava API."""
    url = f"https://www.strava.com/api/v3/activities/" + str(activity_id) + "/streams"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"keys" : keys, "key_by_type": True}
    # valid keys includes ["time", "distance", "latlng", "altitude", "heartrate", "cadence", "watts"]
    # https://developers.strava.com/docs/reference/#api-models-StreamSet
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_gear(access_token, gear_id):
    """Retrieves activities from the Strava API."""
    url = f"https://www.strava.com/api/v3/gear/" + str(gear_id)
    headers = {"Authorization": f"Bearer {access_token}"}
    # params = {"id" : gear_id}
    # valid keys includes ["time", "distance", "latlng", "altitude", "heartrate", "cadence", "watts"]
    # https://developers.strava.com/docs/reference/#api-models-StreamSet
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

In [11]:
def get_latest_starting_coords(db_path):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("""SELECT start_latlng, end_latlng FROM activities 
                WHERE start_latlng is not NULL or 
                length(start_latlng) > 100
                order by start_date desc
                limit 1""")
    rows = cur.fetchone()
    print(rows)
    lat, lon = map(float, rows.split(",")) 
    conn.close()

    return (lat, lon)

get_latest_starting_coords(DB_PATH)

('[]', '[]')


AttributeError: 'tuple' object has no attribute 'split'

## Base functions for storing data in db

#### Initialize Activities Table

In [13]:
def initialize_database():
    """Create the SQLite database and full 'activities' table."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute('''
    CREATE TABLE IF NOT EXISTS activities (
        id INTEGER PRIMARY KEY,
        resource_state INTEGER,
        athlete_id INTEGER,
        athlete_resource_state INTEGER,
        name TEXT,
        distance REAL,
        moving_time INTEGER,
        elapsed_time INTEGER,
        total_elevation_gain REAL,
        type TEXT,
        sport_type TEXT,
        workout_type INTEGER,
        start_date TEXT,
        start_date_local TEXT,
        timezone TEXT,
        utc_offset REAL,
        location_city TEXT,
        location_state TEXT,
        location_country TEXT,
        achievement_count INTEGER,
        kudos_count INTEGER,
        comment_count INTEGER,
        athlete_count INTEGER,
        photo_count INTEGER,
        map_id TEXT,
        map_summary_polyline TEXT,
        map_resource_state INTEGER,
        trainer BOOLEAN,
        commute BOOLEAN,
        manual BOOLEAN,
        private BOOLEAN,
        visibility TEXT,
        flagged BOOLEAN,
        gear_id TEXT,
        start_latlng TEXT,
        end_latlng TEXT,
        average_speed REAL,
        max_speed REAL,
        average_cadence REAL,
        average_watts REAL,
        max_watts INTEGER,
        weighted_average_watts INTEGER,
        device_watts BOOLEAN,
        kilojoules REAL,
        has_heartrate BOOLEAN,
        average_heartrate REAL,
        max_heartrate REAL,
        heartrate_opt_out BOOLEAN,
        display_hide_heartrate_option BOOLEAN,
        elev_high REAL,
        elev_low REAL,
        upload_id INTEGER,
        upload_id_str TEXT,
        external_id TEXT,
        from_accepted_tag BOOLEAN,
        pr_count INTEGER,
        total_photo_count INTEGER,
        has_kudoed BOOLEAN,
        import_date TEXT
    )
    ''')

    conn.commit()
    conn.close()

#### Activities Individual DB Entry Write

In [14]:
import json

def insert_activities(activity_list):
    """Insert activity records, skipping those with duplicate 'id'."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    for activity in activity_list:
        try:
            c.execute('''
            INSERT INTO activities VALUES (
                :id, :resource_state, 
                :athlete_id, :athlete_resource_state,
                :name, :distance, :moving_time, :elapsed_time, :total_elevation_gain,
                :type, :sport_type, :workout_type, :start_date, :start_date_local,
                :timezone, :utc_offset, :location_city, :location_state, :location_country,
                :achievement_count, :kudos_count, :comment_count, :athlete_count, :photo_count,
                :map_id, :map_summary_polyline, :map_resource_state,
                :trainer, :commute, :manual, :private, :visibility, :flagged, :gear_id,
                :start_latlng, :end_latlng,
                :average_speed, :max_speed, :average_cadence, :average_watts,
                :max_watts, :weighted_average_watts, :device_watts, :kilojoules,
                :has_heartrate, :average_heartrate, :max_heartrate,
                :heartrate_opt_out, :display_hide_heartrate_option,
                :elev_high, :elev_low,
                :upload_id, :upload_id_str, :external_id, :from_accepted_tag,
                :pr_count, :total_photo_count, :has_kudoed, :import_date
            )
            ''', {
                "id": activity["id"],
                "resource_state": activity.get("resource_state"),
                "athlete_id": activity.get("athlete", {}).get("id"),
                "athlete_resource_state": activity.get("athlete", {}).get("resource_state"),
                "name": activity.get("name"),
                "distance": activity.get("distance"),
                "moving_time": activity.get("moving_time"),
                "elapsed_time": activity.get("elapsed_time"),
                "total_elevation_gain": activity.get("total_elevation_gain"),
                "type": activity.get("type"),
                "sport_type": activity.get("sport_type"),
                "workout_type": activity.get("workout_type"),
                "start_date": activity.get("start_date"),
                "start_date_local": activity.get("start_date_local"),
                "timezone": activity.get("timezone"),
                "utc_offset": activity.get("utc_offset"),
                "location_city": activity.get("location_city"),
                "location_state": activity.get("location_state"),
                "location_country": activity.get("location_country"),
                "achievement_count": activity.get("achievement_count"),
                "kudos_count": activity.get("kudos_count"),
                "comment_count": activity.get("comment_count"),
                "athlete_count": activity.get("athlete_count"),
                "photo_count": activity.get("photo_count"),
                "map_id": activity.get("map", {}).get("id"),
                "map_summary_polyline": activity.get("map", {}).get("summary_polyline"),
                "map_resource_state": activity.get("map", {}).get("resource_state"),
                "trainer": activity.get("trainer"),
                "commute": activity.get("commute"),
                "manual": activity.get("manual"),
                "private": activity.get("private"),
                "visibility": activity.get("visibility"),
                "flagged": activity.get("flagged"),
                "gear_id": activity.get("gear_id"),
                "start_latlng": json.dumps(activity.get("start_latlng")),
                "end_latlng": json.dumps(activity.get("end_latlng")),
                "average_speed": activity.get("average_speed"),
                "max_speed": activity.get("max_speed"),
                "average_cadence": activity.get("average_cadence"),
                "average_watts": activity.get("average_watts"),
                "max_watts": activity.get("max_watts"),
                "weighted_average_watts": activity.get("weighted_average_watts"),
                "device_watts": activity.get("device_watts"),
                "kilojoules": activity.get("kilojoules"),
                "has_heartrate": activity.get("has_heartrate"),
                "average_heartrate": activity.get("average_heartrate"),
                "max_heartrate": activity.get("max_heartrate"),
                "heartrate_opt_out": activity.get("heartrate_opt_out"),
                "display_hide_heartrate_option": activity.get("display_hide_heartrate_option"),
                "elev_high": activity.get("elev_high"),
                "elev_low": activity.get("elev_low"),
                "upload_id": activity.get("upload_id"),
                "upload_id_str": activity.get("upload_id_str"),
                "external_id": activity.get("external_id"),
                "from_accepted_tag": activity.get("from_accepted_tag"),
                "pr_count": activity.get("pr_count"),
                "total_photo_count": activity.get("total_photo_count"),
                "has_kudoed": activity.get("has_kudoed"),
                "import_date": datetime.now().isoformat()
            })
        except sqlite3.IntegrityError:
            print(f"Skipping duplicate activity with id {activity['id']}")

    conn.commit()
    conn.close()


#### Activities Batch Write

In [15]:
def insert_activities_batch(activity_list, db_path):
    """Efficiently insert multiple activity records into the database."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    data = []
    for activity in activity_list:
        data.append({
            "id": activity["id"],
            "resource_state": activity.get("resource_state"),
            "athlete_id": activity.get("athlete", {}).get("id"),
            "athlete_resource_state": activity.get("athlete", {}).get("resource_state"),
            "name": activity.get("name"),
            "distance": activity.get("distance"),
            "moving_time": activity.get("moving_time"),
            "elapsed_time": activity.get("elapsed_time"),
            "total_elevation_gain": activity.get("total_elevation_gain"),
            "type": activity.get("type"),
            "sport_type": activity.get("sport_type"),
            "workout_type": activity.get("workout_type"),
            "start_date": activity.get("start_date"),
            "start_date_local": activity.get("start_date_local"),
            "timezone": activity.get("timezone"),
            "utc_offset": activity.get("utc_offset"),
            "location_city": activity.get("location_city"),
            "location_state": activity.get("location_state"),
            "location_country": activity.get("location_country"),
            "achievement_count": activity.get("achievement_count"),
            "kudos_count": activity.get("kudos_count"),
            "comment_count": activity.get("comment_count"),
            "athlete_count": activity.get("athlete_count"),
            "photo_count": activity.get("photo_count"),
            "map_id": activity.get("map", {}).get("id"),
            "map_summary_polyline": activity.get("map", {}).get("summary_polyline"),
            "map_resource_state": activity.get("map", {}).get("resource_state"),
            "trainer": activity.get("trainer"),
            "commute": activity.get("commute"),
            "manual": activity.get("manual"),
            "private": activity.get("private"),
            "visibility": activity.get("visibility"),
            "flagged": activity.get("flagged"),
            "gear_id": activity.get("gear_id"),
            "start_latlng": json.dumps(activity.get("start_latlng")),
            "end_latlng": json.dumps(activity.get("end_latlng")),
            "average_speed": activity.get("average_speed"),
            "max_speed": activity.get("max_speed"),
            "average_cadence": activity.get("average_cadence"),
            "average_watts": activity.get("average_watts"),
            "max_watts": activity.get("max_watts"),
            "weighted_average_watts": activity.get("weighted_average_watts"),
            "device_watts": activity.get("device_watts"),
            "kilojoules": activity.get("kilojoules"),
            "has_heartrate": activity.get("has_heartrate"),
            "average_heartrate": activity.get("average_heartrate"),
            "max_heartrate": activity.get("max_heartrate"),
            "heartrate_opt_out": activity.get("heartrate_opt_out"),
            "display_hide_heartrate_option": activity.get("display_hide_heartrate_option"),
            "elev_high": activity.get("elev_high"),
            "elev_low": activity.get("elev_low"),
            "upload_id": activity.get("upload_id"),
            "upload_id_str": activity.get("upload_id_str"),
            "external_id": activity.get("external_id"),
            "from_accepted_tag": activity.get("from_accepted_tag"),
            "pr_count": activity.get("pr_count"),
            "total_photo_count": activity.get("total_photo_count"),
            "has_kudoed": activity.get("has_kudoed"),
            "import_date": datetime.now().isoformat()
        })

    try:
        c.executemany('''
        INSERT OR IGNORE INTO activities VALUES (
            :id, :resource_state, :athlete_id, :athlete_resource_state,
            :name, :distance, :moving_time, :elapsed_time, :total_elevation_gain,
            :type, :sport_type, :workout_type, :start_date, :start_date_local,
            :timezone, :utc_offset, :location_city, :location_state, :location_country,
            :achievement_count, :kudos_count, :comment_count, :athlete_count, :photo_count,
            :map_id, :map_summary_polyline, :map_resource_state,
            :trainer, :commute, :manual, :private, :visibility, :flagged, :gear_id,
            :start_latlng, :end_latlng,
            :average_speed, :max_speed, :average_cadence, :average_watts,
            :max_watts, :weighted_average_watts, :device_watts, :kilojoules,
            :has_heartrate, :average_heartrate, :max_heartrate,
            :heartrate_opt_out, :display_hide_heartrate_option,
            :elev_high, :elev_low,
            :upload_id, :upload_id_str, :external_id, :from_accepted_tag,
            :pr_count, :total_photo_count, :has_kudoed, :import_date
        )
        ''', data)
    except sqlite3.Error as e:
        print("Error inserting batch:", e)

    conn.commit()
    conn.close()


#### Function to Rebuild single activity from flattened version in db

In [16]:
def get_activity_by_id(activity_id):
    """Retrieve a single activity and reconstruct its nested format."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("SELECT * FROM activities WHERE id = ?", (activity_id,))
    row = c.fetchone()
    conn.close()

    if row is None:
        return None

    return {
        "id": row["id"],
        "resource_state": row["resource_state"],
        "athlete": {
            "id": row["athlete_id"],
            "resource_state": row["athlete_resource_state"]
        },
        "name": row["name"],
        "distance": row["distance"],
        "moving_time": row["moving_time"],
        "elapsed_time": row["elapsed_time"],
        "total_elevation_gain": row["total_elevation_gain"],
        "type": row["type"],
        "sport_type": row["sport_type"],
        "workout_type": row["workout_type"],
        "start_date": row["start_date"],
        "start_date_local": row["start_date_local"],
        "timezone": row["timezone"],
        "utc_offset": row["utc_offset"],
        "location_city": row["location_city"],
        "location_state": row["location_state"],
        "location_country": row["location_country"],
        "achievement_count": row["achievement_count"],
        "kudos_count": row["kudos_count"],
        "comment_count": row["comment_count"],
        "athlete_count": row["athlete_count"],
        "photo_count": row["photo_count"],
        "map": {
            "id": row["map_id"],
            "summary_polyline": row["map_summary_polyline"],
            "resource_state": row["map_resource_state"]
        },
        "trainer": bool(row["trainer"]),
        "commute": bool(row["commute"]),
        "manual": bool(row["manual"]),
        "private": bool(row["private"]),
        "visibility": row["visibility"],
        "flagged": bool(row["flagged"]),
        "gear_id": row["gear_id"],
        "start_latlng": json.loads(row["start_latlng"]),
        "end_latlng": json.loads(row["end_latlng"]),
        "average_speed": row["average_speed"],
        "max_speed": row["max_speed"],
        "average_cadence": row["average_cadence"],
        "average_watts": row["average_watts"],
        "max_watts": row["max_watts"],
        "weighted_average_watts": row["weighted_average_watts"],
        "device_watts": bool(row["device_watts"]),
        "kilojoules": row["kilojoules"],
        "has_heartrate": bool(row["has_heartrate"]),
        "average_heartrate": row["average_heartrate"],
        "max_heartrate": row["max_heartrate"],
        "heartrate_opt_out": bool(row["heartrate_opt_out"]),
        "display_hide_heartrate_option": bool(row["display_hide_heartrate_option"]),
        "elev_high": row["elev_high"],
        "elev_low": row["elev_low"],
        "upload_id": row["upload_id"],
        "upload_id_str": row["upload_id_str"],
        "external_id": row["external_id"],
        "from_accepted_tag": bool(row["from_accepted_tag"]),
        "pr_count": row["pr_count"],
        "total_photo_count": row["total_photo_count"],
        "has_kudoed": bool(row["has_kudoed"])
    }


#### Simple Query to get records loaded during current day

In [17]:
def count_activities_imported_today(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT COUNT(*) 
        FROM activities 
        WHERE substr(import_date, 1, 10) = ?
    """, (today_str,))
    
    count = c.fetchone()[0]
    conn.close()
    return count


#### Simple Query to Latest record

In [18]:
def latest_activity_imported(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT * 
        FROM activities 
        WHERE import_date = 
        (select max(import_date) from activities)
    """)
    
    record = c.fetchone()
    conn.close()
    return record

latest_activity_imported(DB_PATH)


(14448198825,
 2,
 24266563,
 1,
 'Morning Walk',
 2917.0,
 2166,
 2225,
 34.0,
 'Walk',
 'Walk',
 None,
 '2025-05-11T15:06:07Z',
 '2025-05-11T10:06:07Z',
 '(GMT-06:00) America/Chicago',
 -18000.0,
 None,
 None,
 None,
 0,
 0,
 0,
 1,
 0,
 'a14448198825',
 '',
 2,
 0,
 0,
 0,
 0,
 'everyone',
 0,
 'g15302697',
 '[]',
 '[]',
 1.347,
 4.8,
 58.9,
 None,
 None,
 None,
 None,
 None,
 1,
 82.7,
 103.0,
 0,
 1,
 275.0,
 260.0,
 15415252303,
 '15415252303',
 '468950890873651212.fit',
 0,
 0,
 0,
 0,
 '2025-05-11T16:30:47.017075')

### Simple Query to get Gear Id from Latest Record

In [19]:
def get_all_gear_ids(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT distinct gear_id 
        FROM activities 
        WHERE gear_id IS NOT NULL
    """)
    
    record = c.fetchall()
    conn.close()
    return record

#### Simple Query to Get Specific record

In [20]:
def activity_data_all(db_path, activity_id):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # Run query and load into DataFrame (with column headers)
    df = pd.read_sql_query(
        "SELECT * FROM activities WHERE id = ?",
        conn,
        params=(activity_id,)
    )
    return df

#### Simple Query to get all Activity IDs

In [21]:
def get_all_activity_ids(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT id
        FROM activities 
        order by start_date desc
              """)
    
    count = c.fetchall()
    conn.close()
    return count

get_all_activity_ids(DB_PATH)

[(14448198825,),
 (14445846595,),
 (14439139788,),
 (14436191500,),
 (14429488624,),
 (14429488648,),
 (14420081833,),
 (14414666612,),
 (14411191781,),
 (14411191925,),
 (14411192865,),
 (14404286770,),
 (14399681906,),
 (14393650080,),
 (14389805238,),
 (14383969194,),
 (14380480511,),
 (14379281304,),
 (14376062303,),
 (14369842769,),
 (14368997811,),
 (14365012542,),
 (14358400377,),
 (14354639209,),
 (14353398775,),
 (14348764058,),
 (14342981460,),
 (14338478098,),
 (14338218423,),
 (14337526946,),
 (14331978461,),
 (14326961057,),
 (14321075456,),
 (14317096914,),
 (14310902427,),
 (14305742622,),
 (14302735322,),
 (14292794382,),
 (14285619866,),
 (14280817149,),
 (14276784576,),
 (14271402828,),
 (14267399463,),
 (14261794985,),
 (14245504574,),
 (14237190879,),
 (14226658058,),
 (14222270286,),
 (14217673520,),
 (14214735620,),
 (14211839733,),
 (14207239257,),
 (14202362392,),
 (14199011311,),
 (14198134902,),
 (14194243688,),
 (14193413852,),
 (14188732734,),
 (14183565736,

#### Simple Query to get all Activity IDs with HR and presumably Streams data

In [22]:
def get_all_activity_ids_with_HR(db_path):
    """Get all activity ids that have HR data, which hopefully will help us avoid a 404 call on the streams data.
    My thinking is that if no HR data, we probably have no streams data at all.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT id
        FROM activities 
        WHERE has_heartrate = 1
        order by start_date desc
              """)
    
    count = c.fetchall()
    conn.close()
    return count

In [23]:
def latest_activity_import_date(db_path):
    """Query for latest activity import_date as a unix timestamp"""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT max(import_date)
        FROM activities 
    """)
    
    record = c.fetchone()[0]
    conn.close()
    dt = datetime.strptime(record, "%Y-%m-%dT%H:%M:%S.%f")
    return int(dt.timestamp())

In [24]:
latest_activity_import_date(DB_PATH)

1746999047

## Database Streams Code

#### Initialize DB  

In [25]:
def initialize_streams_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute("""
    CREATE TABLE IF NOT EXISTS streams (
        activity_id INTEGER PRIMARY KEY,
        
        time_data TEXT,
        time_series_type TEXT,
        time_original_size INTEGER,
        time_resolution TEXT,

        distance_data TEXT,
        distance_series_type TEXT,
        distance_original_size INTEGER,
        distance_resolution TEXT,

        latlng_data TEXT,
        latlng_series_type TEXT,
        latlng_original_size INTEGER,
        latlng_resolution TEXT,

        altitude_data TEXT,
        altitude_series_type TEXT,
        altitude_original_size INTEGER,
        altitude_resolution TEXT,

        velocity_smooth_data TEXT,
        velocity_smooth_series_type TEXT,
        velocity_smooth_original_size INTEGER,
        velocity_smooth_resolution TEXT,

        heartrate_data TEXT,
        heartrate_series_type TEXT,
        heartrate_original_size INTEGER,
        heartrate_resolution TEXT,

        cadence_data TEXT,
        cadence_series_type TEXT,
        cadence_original_size INTEGER,
        cadence_resolution TEXT,

        watts_data TEXT,
        watts_series_type TEXT,
        watts_original_size INTEGER,
        watts_resolution TEXT,

        moving_data TEXT,
        moving_series_type TEXT,
        moving_original_size INTEGER,
        moving_resolution TEXT,

        grade_smooth_data TEXT,
        grade_smooth_series_type TEXT,
        grade_smooth_original_size INTEGER,
        grade_smooth_resolution TEXT
    )
    """)

    conn.commit()
    conn.close()

#### Insert Stream Data for Single Activity ID

In [26]:
def insert_stream_data(activity_id, stream_dict, db_path):
    """
    Inserts or replaces a row in the streams table for a given activity_id.
    stream_dict should have keys like 'time', 'distance', etc., with each value a dict containing:
    {
        'data': [...],
        'series_type': '...',
        'original_size': ...,
        'resolution': '...'
    }
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # Create the column mappings dynamically
    base_columns = []
    placeholders = []
    values = []

    # Always include activity_id
    base_columns.append("activity_id")
    placeholders.append("?")
    values.append(activity_id)

    for key, val in stream_dict.items():
        if not isinstance(val, dict):
            continue  # skip malformed

        base_columns.extend([
            f"{key}_data",
            f"{key}_series_type",
            f"{key}_original_size",
            f"{key}_resolution"
        ])
        placeholders.extend(["?"] * 4)

        values.extend([
            json.dumps(val.get("data")),
            val.get("series_type"),
            val.get("original_size"),
            val.get("resolution")
        ])

    sql = f"""
        INSERT INTO streams ({', '.join(base_columns)})
        VALUES ({', '.join(placeholders)})
    """
    

    try:
        c.execute(sql, values)
        conn.commit()
        conn.close()
    except sqlite3.IntegrityError:
        print(f"Activity {activity_id} already exists in the 'streams' table. Skipping insert.")

#### Simple Query to Get all activity IDs from Streams

In [27]:
def get_all_activity_ids_streams(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT activity_id
        FROM streams 
              """)
    
    count = c.fetchall()
    conn.close()
    return count

In [18]:
#### Simple Query to Get all Streams for latest activity

In [28]:
def single_stream_example(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    
    # Run query and load into DataFrame (with column headers)
    df = pd.read_sql_query(
        """
        SELECT * 
        FROM streams 
        ORDER BY activity_id desc
        limit 1
    """,
        conn
    )
    return df


In [20]:
test_df = single_stream_example(DB_PATH)
test_df 

,activity_id,time_data,time_series_type,time_original_size,time_resolution,distance_data,distance_series_type,distance_original_size,distance_resolution,latlng_data,latlng_series_type,latlng_original_size,latlng_resolution,altitude_data,altitude_series_type,altitude_original_size,altitude_resolution,velocity_smooth_data,velocity_smooth_series_type,velocity_smooth_original_size,velocity_smooth_resolution,heartrate_data,heartrate_series_type,heartrate_original_size,heartrate_resolution,cadence_data,cadence_series_type,cadence_original_size,cadence_resolution,watts_data,watts_series_type,watts_original_size,watts_resolution,moving_data,moving_series_type,moving_original_size,moving_resolution,grade_smooth_data,grade_smooth_series_type,grade_smooth_original_size,grade_smooth_resolution
0,14393650080,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",distance,3049,high,"[0.0, 0.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0...",distance,3049,high,"[[44.949551, -93.324906], [44.949551, -93.3249...",distance,3049,high,"[278.0, 278.0, 278.0, 278.0, 278.0, 278.0, 278...",distance,3049,high,"[0.0, 0.0, 0.0, 0.667, 1.0, 1.2, 1.6, 2.0, 2.0...",distance,3049,high,"[61, 61, 61, 61, 61, 61, 62, 63, 63, 63, 63, 6...",distance,3049,high,"[86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 87, 8...",distance,3049,high,"[null, null, null, null, null, null, null, nul...",distance,3049,high,"[false, false, false, true, true, true, true, ...",distance,3049,high,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",distance,3049,high


In [21]:
len(test_df.time_data[0])

17184

In [22]:
len(test_df.heartrate_data[0])

15197

In [24]:
def get_streams_data(activity_id, db_path):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT distance_data, heartrate_data, altitude_data FROM streams WHERE activity_id = ?", (activity_id,))
    row = cur.fetchone()
    conn.close()

    if row:
        try:
            distance = json.loads(row[0]) if row[0] else []
            heartrate = json.loads(row[1]) if row[1] else []
            altitude = json.loads(row[2]) if row[2] else []
            return distance, heartrate, altitude
        except Exception as e:
            print("Failed to load stream data:", e)
    return [], [], []

In [107]:
z = get_streams_data(14414666612, DB_PATH)

OperationalError: no such column: id

In [105]:
z

([], [], [])

## Database Gear Code

#### Initialize Gear Table  

In [242]:
def drop_gear_table():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute("""
DROP TABLE gear""")
    conn.commit()
    conn.close()

drop_gear_table()

In [29]:
def initialize_gear_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute("""
    CREATE TABLE IF NOT EXISTS gear (
        gear_id TEXT PRIMARY KEY,
        is_primary BOOLEAN,
        nickname TEXT,
        resource_state INTEGER,
        retired BOOLEAN,
        distance INTEGER,
        brand_name TEXT,
        model_name TEXT,      
        frame_type INTEGER,
        description TEXT,
        weight REAL
    )
    """)

    conn.commit()
    conn.close()

#### Insert Single Gear

In [30]:
import json

def insert_single_gear(gear, db_path):
    """Insert activity records, skipping those with duplicate 'id'."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    try:
        c.execute('''
        INSERT OR REPLACE INTO gear VALUES (
            :gear_id, :is_primary, :nickname, 
            :resource_state, :retired, :distance,
            :brand_name, :model_name,
            :frame_type, :description, :weight
                  )
        ''', {
            "gear_id": gear.get("id"),
            "is_primary": gear.get("primary"),
            "nickname" : gear.get("nickname"),
            "resource_state": gear.get("resource_state"),
            "retired" : gear.get("retired"),
            "distance": gear.get("distance"),
            "brand_name": gear.get("brand_name"),
            "model_name": gear.get("model_name"),
            "frame_type": gear.get("frame_type"),
            "description": gear.get("description"),
            "weight" : gear.get("weight"),
            "import_date": datetime.now().isoformat()
        })
    except sqlite3.IntegrityError:
        print(f"Skipping duplicate activity with id {gear['id']}")

    conn.commit()
    conn.close()

#### Get All Gear

In [31]:
def get_all_gear(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(
        """
        SELECT *
        FROM gear 
        limit 5
    """,
        conn
    )
    return df
   

In [249]:
get_all_gear(DB_PATH)

,gear_id,is_primary,nickname,resource_state,retired,distance,brand_name,model_name,frame_type,description,weight
0,b5036222,0,CTS Road,3,1,4390192,Tommaso,Imola,3.0,Stolen,23.0
1,g4006462,0,CTS M860 x1,3,1,687401,New Balance,M860 v9,NaN,Mens 10.5 M860 BK9 Black/Grey,NaN
2,g4556561,0,CTS M860 x2,3,1,676902,New Balance,M860 v9,NaN,Mens 10.5 M860 BK9 Black/Grey,NaN
3,g4872382,0,CTS M860 x3,3,1,694916,New Balance,M860 v9,NaN,Mens 10.5 M860 BK9 Green/Grey,NaN
4,b6893678,0,CTS Road v2,3,0,6487523,Bianchi,"""C2C"" VIA NIRONE Alu Sora 9sp mix Compact",3.0,Acquired 4.1.2020,21.7


In [32]:
def get_gearid_for_specific_activity(db_path, activity_id):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    c.execute(
        """
        SELECT gear_id
        FROM activities 
        where id = ?
    """, (activity_id,)
    )
    count = c.fetchone()[0]
    conn.close()
    return count

In [85]:
get_gearid_for_specific_activity(DB_PATH, 14436191500)

'g15852452'

## Get Latest Activity and Associated Streams

In [14]:
try:
    activities = []
    streams = []
    page = 1
    per_page = 1
    while True:
        activities_page = get_activities(AUTH_TOKEN, page=page, per_page=per_page)
        if not activities_page:
            break
        activities.extend(activities_page)
        page += 1
        time.sleep(1)  # Respect rate limits, adjust if needed
        break

    # Process the activity data
    for activity in activities:
        print(f"Activity ID: {activity['id']}, Name: {activity['name']}, Type: {activity['type']}")

    keys = ["time", "distance", "latlng", "altitude", "heartrate", "cadence", "watts"]
    keys = "time, latlng"
    # Process stream data
    for a in activities:
        stream = get_streams(AUTH_TOKEN, keys, a['id'])
        if not stream:
            break
        streams.extend(stream)
        time.sleep(1)  # Respect rate limits, adjust if needed
        break

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Activity ID: 14404286770, Name: Morning Run, Type: Run


## Bulk Historical Processing of Activities
#### Run this up to 1000 API calls per day during historical data loading
#### Quit IMMEDIATELY if rate limited

In [33]:
# starting point, build unix timestamps in 6 month batches

# time_0 = time.mktime(datetime.datetime(2019, 1, 1, 0, 0, 0).timetuple())
# time_1 = time.mktime(datetime.datetime(2019, 7, 1, 0, 0, 0).timetuple()) 
# time_2 = time.mktime(datetime.datetime(2020, 1, 1, 0, 0, 0).timetuple()) 
# time_3 = time.mktime(datetime.datetime(2020, 7, 1, 0, 0, 0).timetuple())
# time_4 = time.mktime(datetime.datetime(2021, 1, 1, 0, 0, 0).timetuple())
# time_5 = time.mktime(datetime.datetime(2021, 7, 1, 0, 0, 0).timetuple())
# time_6 = time.mktime(datetime.datetime(2022, 1, 1, 0, 0, 0).timetuple())
# time_7 = time.mktime(datetime.datetime(2022, 7, 1, 0, 0, 0).timetuple())
# time_8 = time.mktime(datetime.datetime(2023, 1, 1, 0, 0, 0).timetuple())
# time_9 = time.mktime(datetime.datetime(2023, 7, 1, 0, 0, 0).timetuple())
# time_10 = time.mktime(datetime.datetime(2024, 1, 1, 0, 0, 0).timetuple())
# time_11 = time.mktime(datetime.datetime(2024, 7, 1, 0, 0, 0).timetuple())
# time_12 = time.mktime(datetime.datetime(2025, 1, 1, 0, 0, 0).timetuple())
# time_13 = time.mktime(datetime.datetime(2025, 7, 1, 0, 0, 0).timetuple())

day_by_day_before = time.mktime(datetime(2025, 5, 15, 0, 0, 0).timetuple())
day_by_day_after = time.mktime(datetime(2025, 5, 12, 0, 0, 0).timetuple())

In [105]:
print(day_by_day_after)

1746853200.0


In [37]:
# ensure db table exists
initialize_database()

# Get the data
try:
    activities = []
    page = 1
    per_page = 30
    while True:
        activities_page = get_activities(ACCESS_TOKEN, page=page, per_page=per_page, before = day_by_day_before, after = day_by_day_after) # currently done, latest after time was time_12
        if not activities_page:
            break
        print(len(activities_page)) # expect 30 each time unless final page
        activities.extend(activities_page)
        
        insert_activities_batch(activities_page, DB_PATH) # attempt to bulk write to db
        time.sleep(1)  # Respect rate limits, adjust if needed

        num_today = count_activities_imported_today(DB_PATH) # count objects in db now
        print(f"{num_today} activities were imported so far.")

        page += 1
        time.sleep(1)  # Respect rate limits, adjust if needed
        if page > 10:
            break

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

num_today = count_activities_imported_today(DB_PATH)
print(f"{num_today} activities were imported so far today")
print(f"{len(activities)} activities were imported from the API this round.")

4
4 activities were imported so far.
4 activities were imported so far today
4 activities were imported from the API this round.


## Bulk Historical Processing of Data Streams and Gear
#### Run this every 15 minutes up to 10 times a day during historical data loading

In [44]:
# Initialize the DB
initialize_streams_db()

# get all stream activity IDs
stream_activity_ids = get_all_activity_ids_streams("strava_data.db")
stream_activity_ids = set(i[0] for i in stream_activity_ids)

# get all activity ID's in db sorted by activity date desc as set
sorted_activity_list = get_all_activity_ids_with_HR("strava_data.db")
sorted_activity_list = set(i[0] for i in sorted_activity_list)

# Take the 
valid_activity_ids = tuple(sorted_activity_list - stream_activity_ids)

keys = "time,distance,latlng,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth"

for i, activity_integer in enumerate(valid_activity_ids):
    
    # attempt to pull stream data
    try:
        stream = get_streams(ACCESS_TOKEN, keys, activity_integer)
        if not stream:
            print('no stream')
            continue
    
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        if "429" in str(e):
            print("RATE LIMITED!!!")
            break
        if "404" in str(e):
            print("Stream data unavailable for activity")
            continue
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    time.sleep(1)  # Respect rate limits, adjust if needed
    insert_stream_data(activity_integer, stream, db_path=DB_PATH)
    time.sleep(1)  # Respect rate limits, adjust if needed

    # attempt to pull gear data

    try:
        print(activity_integer)
        shoe = get_gearid_for_specific_activity(DB_PATH, activity_integer)
        print(shoe)
        shoe_data = get_gear(ACCESS_TOKEN, shoe)
        if not stream:
            print('no shoe_data')
            continue
    
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        if "429" in str(e):
            print("RATE LIMITED!!!")
            break
        if "404" in str(e):
            print("Stream data unavailable for activity")
            continue
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    time.sleep(1)  # Respect rate limits, adjust if needed
    insert_single_gear(shoe_data, db_path=DB_PATH)
    time.sleep(1)  # Respect rate limits, adjust if needed

    if i > 10:
        break



In [72]:
num_today = count_activities_imported_today(DB_PATH)
print(f"{len(get_all_activity_ids_streams("strava_data.db"))} streams in the db.")
print(f"{len(get_all_activity_ids_with_HR("strava_data.db"))} valid activities with potential streams data exist in the db.")
print(f"{len(get_all_activity_ids_with_HR("strava_data.db")) - len(get_all_activity_ids_streams("strava_data.db"))} to go!")

2753 streams in the db.
2751 valid activities with potential streams data exist in the db.
-2 to go!


## Gear Workspace

#### The below fully rebuilds the gear table

In [40]:
# Initialize the DB
initialize_gear_db()

shoe_id = get_all_gear_ids(DB_PATH)

for id in shoe_id:
    shoe = get_gear(ACCESS_TOKEN, id[0])
    insert_single_gear(shoe, db_path=DB_PATH)


## Airflow Section

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
from datetime import timedelta
import requests
import json

CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
REFRESH_TOKEN = os.environ.get("REFRESH_TOKEN")
DB_PATH = os.environ.get("DATABASE")

def latest_activity_import_date(db_path):
    """Query for latest activity import_date as a unix timestamp"""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT max(import_date)
        FROM activities 
    """)
    
    record = c.fetchone()[0]
    conn.close()
    dt = datetime.strptime(record, "%Y-%m-%dT%H:%M:%S.%f")
    return int(dt.timestamp())

def get_activities(access_token, page=1, per_page=30, **optional_parameters):
    """Retrieves activities from the Strava API.
    Optional parameters should be provided at the end of the call like so:
    before = epoch_timestamp, after = epoch_timestamp
    """
    url = f"https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"page": page, "per_page": per_page}
    params.update(optional_parameters)
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_streams(access_token, keys = "time,distance,latlng,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth", activity_id):
    """Retrieves activities from the Strava API."""
    url = f"https://www.strava.com/api/v3/activities/" + str(activity_id) + "/streams"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"keys" : keys, "key_by_type": True}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_gear(access_token, gear_id):
    """Retrieves activities from the Strava API."""
    url = f"https://www.strava.com/api/v3/gear/" + str(gear_id)
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def insert_activities_batch(activity_list, db_path = DB_PATH):
    """Efficiently insert multiple activity records into the database."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    data = []
    for activity in activity_list:
        data.append({
            "id": activity["id"],
            "resource_state": activity.get("resource_state"),
            "athlete_id": activity.get("athlete", {}).get("id"),
            "athlete_resource_state": activity.get("athlete", {}).get("resource_state"),
            "name": activity.get("name"),
            "distance": activity.get("distance"),
            "moving_time": activity.get("moving_time"),
            "elapsed_time": activity.get("elapsed_time"),
            "total_elevation_gain": activity.get("total_elevation_gain"),
            "type": activity.get("type"),
            "sport_type": activity.get("sport_type"),
            "workout_type": activity.get("workout_type"),
            "start_date": activity.get("start_date"),
            "start_date_local": activity.get("start_date_local"),
            "timezone": activity.get("timezone"),
            "utc_offset": activity.get("utc_offset"),
            "location_city": activity.get("location_city"),
            "location_state": activity.get("location_state"),
            "location_country": activity.get("location_country"),
            "achievement_count": activity.get("achievement_count"),
            "kudos_count": activity.get("kudos_count"),
            "comment_count": activity.get("comment_count"),
            "athlete_count": activity.get("athlete_count"),
            "photo_count": activity.get("photo_count"),
            "map_id": activity.get("map", {}).get("id"),
            "map_summary_polyline": activity.get("map", {}).get("summary_polyline"),
            "map_resource_state": activity.get("map", {}).get("resource_state"),
            "trainer": activity.get("trainer"),
            "commute": activity.get("commute"),
            "manual": activity.get("manual"),
            "private": activity.get("private"),
            "visibility": activity.get("visibility"),
            "flagged": activity.get("flagged"),
            "gear_id": activity.get("gear_id"),
            "start_latlng": json.dumps(activity.get("start_latlng")),
            "end_latlng": json.dumps(activity.get("end_latlng")),
            "average_speed": activity.get("average_speed"),
            "max_speed": activity.get("max_speed"),
            "average_cadence": activity.get("average_cadence"),
            "average_watts": activity.get("average_watts"),
            "max_watts": activity.get("max_watts"),
            "weighted_average_watts": activity.get("weighted_average_watts"),
            "device_watts": activity.get("device_watts"),
            "kilojoules": activity.get("kilojoules"),
            "has_heartrate": activity.get("has_heartrate"),
            "average_heartrate": activity.get("average_heartrate"),
            "max_heartrate": activity.get("max_heartrate"),
            "heartrate_opt_out": activity.get("heartrate_opt_out"),
            "display_hide_heartrate_option": activity.get("display_hide_heartrate_option"),
            "elev_high": activity.get("elev_high"),
            "elev_low": activity.get("elev_low"),
            "upload_id": activity.get("upload_id"),
            "upload_id_str": activity.get("upload_id_str"),
            "external_id": activity.get("external_id"),
            "from_accepted_tag": activity.get("from_accepted_tag"),
            "pr_count": activity.get("pr_count"),
            "total_photo_count": activity.get("total_photo_count"),
            "has_kudoed": activity.get("has_kudoed"),
            "import_date": datetime.now().isoformat()
        })

    try:
        c.executemany('''
        INSERT OR IGNORE INTO activities VALUES (
            :id, :resource_state, :athlete_id, :athlete_resource_state,
            :name, :distance, :moving_time, :elapsed_time, :total_elevation_gain,
            :type, :sport_type, :workout_type, :start_date, :start_date_local,
            :timezone, :utc_offset, :location_city, :location_state, :location_country,
            :achievement_count, :kudos_count, :comment_count, :athlete_count, :photo_count,
            :map_id, :map_summary_polyline, :map_resource_state,
            :trainer, :commute, :manual, :private, :visibility, :flagged, :gear_id,
            :start_latlng, :end_latlng,
            :average_speed, :max_speed, :average_cadence, :average_watts,
            :max_watts, :weighted_average_watts, :device_watts, :kilojoules,
            :has_heartrate, :average_heartrate, :max_heartrate,
            :heartrate_opt_out, :display_hide_heartrate_option,
            :elev_high, :elev_low,
            :upload_id, :upload_id_str, :external_id, :from_accepted_tag,
            :pr_count, :total_photo_count, :has_kudoed, :import_date
        )
        ''', data)
    except sqlite3.Error as e:
        print("Error inserting batch:", e)

    conn.commit()
    conn.close()

def insert_stream_data(activity_id, stream_dict, db_path = DB_PATH):
    """
    Inserts or replaces a row in the streams table for a given activity_id.
    stream_dict should have keys like 'time', 'distance', etc., with each value a dict containing:
    {
        'data': [...],
        'series_type': '...',
        'original_size': ...,
        'resolution': '...'
    }
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # Create the column mappings dynamically
    base_columns = []
    placeholders = []
    values = []

    # Always include activity_id
    base_columns.append("activity_id")
    placeholders.append("?")
    values.append(activity_id)

    for key, val in stream_dict.items():
        if not isinstance(val, dict):
            continue  # skip malformed

        base_columns.extend([
            f"{key}_data",
            f"{key}_series_type",
            f"{key}_original_size",
            f"{key}_resolution"
        ])
        placeholders.extend(["?"] * 4)

        values.extend([
            json.dumps(val.get("data")),
            val.get("series_type"),
            val.get("original_size"),
            val.get("resolution")
        ])

    sql = f"""
        INSERT INTO streams ({', '.join(base_columns)})
        VALUES ({', '.join(placeholders)})
    """
    

    try:
        c.execute(sql, values)
        conn.commit()
        conn.close()
    except sqlite3.IntegrityError:
        print(f"Activity {activity_id} already exists in the 'streams' table. Skipping insert.")

def insert_single_gear(gear, db_path=DB_PATH):
    """Insert activity records, skipping those with duplicate 'id'."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    try:
        c.execute('''
        INSERT OR REPLACE INTO gear VALUES (
            :gear_id, :is_primary, :nickname, 
            :resource_state, :retired, :distance,
            :brand_name, :model_name,
            :frame_type, :description, :weight
                  )
        ''', {
            "gear_id": gear.get("id"),
            "is_primary": gear.get("primary"),
            "nickname" : gear.get("nickname"),
            "resource_state": gear.get("resource_state"),
            "retired" : gear.get("retired"),
            "distance": gear.get("distance"),
            "brand_name": gear.get("brand_name"),
            "model_name": gear.get("model_name"),
            "frame_type": gear.get("frame_type"),
            "description": gear.get("description"),
            "weight" : gear.get("weight"),
            "import_date": datetime.now().isoformat()
        })
    except sqlite3.IntegrityError:
        print(f"Skipping duplicate activity with id {gear['id']}")

    conn.commit()
    conn.close()

# This needs to happen first and everything else will fail if this fails
def refresh_access_token(client_id, client_secret, refresh_token):
    """API call using local refresh token to get new access token.
    Needs to pull client_id, client_secret, and refresh_token from environment variables.
    Needs to write new refresh token to environment variables and pass on the received access
    token to the following api functions
    """
    response = requests.post(
        url="https://www.strava.com/oauth/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
        },
    )
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    os.environ["REFRESH_TOKEN"] = response["refresh_token"]
    kwargs['ti'].xcom_push(key='access_token', value=response.json()["access_token"])
    
def fetch_activity_data_and_write_to_db(**kwargs):
    """Function to call the strava api for latest activity data since that last import_date.
    Needs to accept an before and after unix timestamp as a parameter as well as db path
    to query for the latest import_date.
    """
    after = latest_activity_import_date(db_path)
    before = after + 86400
    page = 1
    per_page = 30
    try:
        activities_page = get_activities(ACCESS_TOKEN, page=page, per_page=per_page, before = before, after = after)     
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decode error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    insert_activities_batch(activities_page)
    kwargs['ti'].xcom_push(key='activity_dict', value=activities_page)


def fetch_streams_data_and_write_to_db(**kwargs):
    """Function to call the strava api for stream data related to activity id's pulled in from 
    fetch activity_data then write single stream datasets to the db.
    """
    for activity in activities_page:
        try:
            stream = get_streams(ACCESS_TOKEN, keys, int(activity[id]))
            if not stream:
                print('no stream')
                continue
        
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            if "429" in str(e):
                print("RATE LIMITED!!!")
                break
            if "404" in str(e):
                print("Stream data unavailable for activity")
                continue
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        time.sleep(1)  # Respect rate limits, adjust if needed
        insert_stream_data(activity[id], stream)

def fetch_gear_data_and_write_to_db(**kwargs):
    """Function to call the strava api for single gear data related to gear_id pulled in from 
    fetch activity_data then write single gear update to the db.
    """
    for activity in activities_page:
        try:
            shoe = activity['gear_id']
            if not shoe:
                print('no shoe')
                continue
        
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            if "429" in str(e):
                print("RATE LIMITED!!!")
                break
            if "404" in str(e):
                print("Stream data unavailable for activity")
                continue
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        time.sleep(1)  # Respect rate limits, adjust if needed
        insert_single_gear(shoe)


default_args = {
    'owner': 'you',
    'depends_on_past': False,
    'email_on_failure': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    'api_to_db_pipeline',
    default_args=default_args,
    description='Fetch Strava API Activity data and update 3 db tables',
    schedule_interval='30 7,17,19 * * *',
    start_date=days_ago(1),
    catchup=False,
    tags=['example'],
) as dag:

    refresh_tokens = PythonOperator(
        task_id='refresh_access_token',
        python_callable=refresh_access_token,
        provide_context=True,
    )

    update_activities = PythonOperator(
        task_id='fetch_and_write_activities',
        python_callable=fetch_activity_data_and_write_to_db,
        provide_context=True,
    )

    update_streams = PythonOperator(
        task_id='fetch_and_write_streams',
        python_callable=fetch_streams_data_and_write_to_db,
        provide_context=True,
    )

    update_gear = PythonOperator(
    task_id='fetch_and_write_gear',
    python_callable=fetch_gear_data_and_write_to_db,
    provide_context=True,
    )


    refresh_tokens >> [update_activities, update_streams, update_gear]
